In [1]:
import itertools
import os
import random

In [4]:
# Positions in the DUX4 protein sequence that are critical for DNA binding
critical_positions = [47, 50, 51, 209, 212, 213]
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
known_critical_residues = {47: 'R', 50: 'Q', 51: 'N', 209: 'R', 212: 'Q', 213: 'N'}

# Function to generate single-point mutations
def generate_single_mutations(critical_positions, known_residues, amino_acids):
    single_mutations = []
    for pos in critical_positions:
        original_residue = known_residues[pos]
        for aa in amino_acids:
            if aa != original_residue:  # Skip the original residue
                mutation = known_residues.copy()
                mutation[pos] = aa
                single_mutations.append(tuple(mutation[pos] for pos in critical_positions))
    return single_mutations

# Generate all single amino acid mutations
single_mutations = generate_single_mutations(critical_positions, known_critical_residues, amino_acids)

# Validate the permutations
def is_valid(permutation):
    # Known critical residues for the positions
    known_critical_residues = {47: 'R', 50: 'Q', 51: 'N', 209: 'R', 212: 'Q', 213: 'N'}
    
    # Check if permutation matches known critical residues
    for idx, pos in enumerate(critical_positions):
        if pos in known_critical_residues and permutation[idx] != known_critical_residues[pos]:
            return False
    
    # Avoiding certain rare or structurally unfavorable amino acids
    unfavorable_residues = {'C', 'W'}  # Example of residues to avoid
    if any(residue in unfavorable_residues for residue in permutation):
        return False
    
    # Avoid excessive hydrophobic residues which could disrupt binding
    hydrophobic_residues = {'A', 'I', 'L', 'M', 'F', 'W', 'V'}
    if sum(1 for residue in permutation if residue in hydrophobic_residues) > 2:
        return False

    # Ensure a balance of charged residues to maintain solubility and binding
    charged_residues = {'R', 'K', 'D', 'E'}
    if sum(1 for residue in permutation if residue in charged_residues) < 1:
        return False
    
    # Avoid combinations that could introduce steric clashes
    bulky_residues = {'F', 'Y', 'W'}
    if sum(1 for residue in permutation if residue in bulky_residues) > 1:
        return False
    
    return True

# Prune the permutations
pruned_permutations = [perm for perm in single_mutations if is_valid(perm)]

# Print the pruned permutations for verification
print(pruned_permutations)

[]


In [ ]:

original_sequence = "MQSDSQGLQKTLGNGYSYLFKLRDSAAGSHRYWNSYDKVEGFVKMLQKAGDTVGKFTETSAKLSDFSRVFQGPRDGSILKSGAERTRLLRQMSRFVKG"

def mutate_sequence(sequence, positions, mutations):
    sequence = list(sequence)
    for pos, mut in zip(positions, mutations):
        sequence[pos] = mut
    return ''.join(sequence)

# Prepare AlphaFold input files
def prepare_alphafold_input(sequence, output_dir):
    fasta_content = f">DUX4_DBD\n{sequence}\n"
    fasta_path = os.path.join(output_dir, "input.fasta")
    with open(fasta_path, "w") as fasta_file:
        fasta_file.write(fasta_content)
    return fasta_path

# Function to run AlphaFold prediction
def run_alphafold(fasta_path, output_dir):
    os.system(f"run_alphafold.sh -i {fasta_path} -o {output_dir}")

results = []
output_base_dir = "/path/to/output_dir"  # Set this to your desired output directory

for perm in pruned_permutations:
    mutated_sequence = mutate_sequence(original_sequence, critical_positions, perm)
    output_dir = os.path.join(output_base_dir, f"perm_{'_'.join(perm)}")
    os.makedirs(output_dir, exist_ok=True)
    fasta_path = prepare_alphafold_input(mutated_sequence, output_dir)
    run_alphafold(fasta_path, output_dir)
    results.append((mutated_sequence, output_dir))


In [ ]:
import os
import random

# Placeholder function to analyze binding strength
def analyze_binding_strength(output_dir):
    # Implement the analysis of PDB files
    # For demonstration, let's assume we parse a PDB file and calculate a binding strength score
    pdb_file = os.path.join(output_dir, 'binding_model.pdb')
    if not os.path.exists(pdb_file):
        raise FileNotFoundError(f"{pdb_file} not found")
    
    # Example binding strength calculation (replace with actual logic)
    binding_strength_score = random.uniform(0, 100)  # Replace with actual calculation
    return binding_strength_score

# Placeholder function to screen for off-target effects
def screen_off_target(binding_strength):
    # Implement the screening logic
    # For demonstration, let's assume we use the binding strength to derive an off-target score
    # Example off-target score calculation (replace with actual logic)
    off_target_score = random.uniform(0, 10) / binding_strength  # Replace with actual calculation
    return off_target_score

# Assuming 'results' is a list of tuples where each tuple is (sequence, output_directory)
# Example placeholder results
results = [
    (('R', 'Q', 'N', 'R', 'Q', 'N'), 'output_dir1'),
    (('R', 'Q', 'N', 'A', 'Q', 'N'), 'output_dir2'),
    # Add more sequences and corresponding output directories as needed
]

analyzed_results = []
for seq, out_dir in results:
    try:
        binding_strength = analyze_binding_strength(out_dir)
        off_target_score = screen_off_target(binding_strength)
        analyzed_results.append((seq, binding_strength, off_target_score))
    except FileNotFoundError as e:
        print(e)  # Handle missing PDB files gracefully

# Sort and select the best permutations
sorted_results = sorted(analyzed_results, key=lambda x: (x[1], -x[2]))  # Prioritize strong binding and low off-target score
best_candidates = sorted_results[:10]  # Top 10 candidates

# Print the best candidates for verification
for candidate in best_candidates:
    print(candidate)
